In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Image segmentation with a U-Net-like architecture

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2019/03/20<br>
**Last modified:** 2020/04/20<br>
**Description:** Image segmentation model trained from scratch on the Oxford Pets dataset.

## Download the data

In [ ]:
IMAGE_PATH = "/content/drive/MyDrive/Openclassrooms/p8/P8_Cityscapes_leftImg8bit_trainvaltest.zip"
LABEL_PATH = "/content/drive/MyDrive/Openclassrooms/p8/P8_Cityscapes_gtFine_trainvaltest.zip"

In [ ]:
#!unzip -q {IMAGE_PATH} -d /content/drive/MyDrive/Openclassrooms/p8
#!unzip -q {LABEL_PATH} -d /content/drive/MyDrive/Openclassrooms/p8

## Prepare paths of input images and target segmentation masks

In [ ]:
import os

input_dir = "/content/drive/MyDrive/Openclassrooms/p8/leftImg8bit"
target_dir = "/content/drive/MyDrive/Openclassrooms/p8/gtFine"
img_size = (160, 160)
num_classes = 3
batch_size = 32

In [ ]:
directory = ['val', 'test', 'train']

In [ ]:
import os
import shutil
import traceback

def images_in_one_folder(base_dir, directories):
    for dir in directories:
        print(f"Directory: {dir}")
        folder_path = os.path.join(base_dir, dir)

        try:
            # Vérifier si le chemin existe
            if not os.path.exists(folder_path):
                print(f"Le chemin {folder_path} n'existe pas.")
                continue

            # Parcourir chaque sous-dossier dans le dossier courant (val, test, train)
            for folder in os.listdir(folder_path):
                folder_full_path = os.path.join(folder_path, folder)

                try:
                    # Vérifier si c'est un dossier
                    if os.path.isdir(folder_full_path):
                        # Parcourir chaque fichier dans le sous-dossier
                        for file in os.listdir(folder_full_path):
                            file_full_path = os.path.join(folder_full_path, file)

                            try:
                                # Déplacer le fichier dans le dossier parent
                                shutil.move(file_full_path, folder_path)
                                print(f"Fichier déplacé: {file_full_path} vers {folder_path}")
                            except Exception as e:
                                print(f"Erreur lors du déplacement du fichier {file_full_path}: {e}")
                                traceback.print_exc()

                        try:
                            # Supprimer le sous-dossier vide
                            os.rmdir(folder_full_path)
                            print(f"Dossier supprimé: {folder_full_path}")
                        except Exception as e:
                            print(f"Erreur lors de la suppression du dossier {folder_full_path}: {e}")
                            traceback.print_exc()
                except Exception as e:
                    print(f"Erreur lors du traitement du dossier {folder_full_path}: {e}")
                    traceback.print_exc()
        except Exception as e:
            print(f"Erreur lors du traitement du répertoire {dir}: {e}")
            traceback.print_exc()

    print("Tous les fichiers ont été déplacés.\n")

# Chemins des répertoires
input_dir = "/content/drive/MyDrive/Openclassrooms/p8/leftImg8bit"
target_dir = "/content/drive/MyDrive/Openclassrooms/p8/gtFine"

# Liste des répertoires à traiter
directories = ['val', 'test', 'train']

# Appel de la fonction pour les deux répertoires
images_in_one_folder(input_dir, directories)
images_in_one_folder(target_dir, directories)


In [ ]:
import os

def clean_directories(base_dir, directories, tag):
    """
    Parcourt les dossiers spécifiés et supprime les fichiers qui ne contiennent pas le tag donné.

    :param base_dir: Le répertoire de base contenant les sous-dossiers à nettoyer.
    :param directories: Une liste des sous-dossiers à parcourir (par exemple, ['train', 'test', 'val']).
    :param tag: Le tag que les fichiers doivent contenir pour ne pas être supprimés.
    """
    for dir in directories:
        print(f"Directory: {dir}")
        folder_path = os.path.join(base_dir, dir)

        try:
            # Vérifier si le chemin existe
            if not os.path.exists(folder_path):
                print(f"Le chemin {folder_path} n'existe pas.")
                continue

            # Parcourir chaque fichier dans le dossier courant
            for file in os.listdir(folder_path):
                file_full_path = os.path.join(folder_path, file)

                try:
                    # Vérifier si c'est un fichier et s'il ne contient pas le tag
                    if os.path.isfile(file_full_path) and tag not in file:
                        # Supprimer le fichier
                        os.remove(file_full_path)
                        print(f"Fichier supprimé: {file_full_path}")
                except Exception as e:
                    print(f"Erreur lors du traitement du fichier {file_full_path}: {e}")
                    traceback.print_exc()

        except Exception as e:
            print(f"Erreur lors du traitement du répertoire {dir}: {e}")
            traceback.print_exc()

    print("Nettoyage terminé.\n")

# Chemins des répertoires
target_dir = "/content/drive/MyDrive/Openclassrooms/p8/gtFine"

# Liste des répertoires à traiter
directories = ['val', 'test', 'train']

# Tag à rechercher dans les noms de fichiers
tag = 'gtFine_color'

# Appel de la fonction pour nettoyer les répertoires
clean_directories(target_dir, directories, tag)


In [ ]:
# What does one input image and corresponding segmentation mask look like?
from IPython.display import Image, display
from keras.utils import load_img
from PIL import ImageOps

input_img_paths = sorted([os.path.join(input_dir, 'train', img) for img in os.listdir(os.path.join(input_dir, 'train'))])
target_img_paths = sorted([os.path.join(target_dir, 'train', img) for img in os.listdir(os.path.join(target_dir, 'train'))])

print(input_img_paths)
print(target_img_paths)



In [ ]:
# Display input image #9
display(Image(filename=input_img_paths[11]))

from IPython.display import Image, display
from keras.utils import load_img
from PIL import ImageOps
import os

# Assurez-vous que target_img_paths est bien défini et contient les chemins corrects
input_dir = "/content/drive/MyDrive/Openclassrooms/p8/leftImg8bit"
target_dir = "/content/drive/MyDrive/Openclassrooms/p8/gtFine"

# Liste des fichiers dans le dossier target
target_img_paths = sorted([os.path.join(target_dir, 'train', img) for img in os.listdir(os.path.join(target_dir, 'train'))])

# Vérifiez que le chemin existe et est correct
print(f"Chemin du fichier 11: {target_img_paths[11]}")

# Vérifiez si le fichier existe
if os.path.exists(target_img_paths[11]):
    # Affichez l'image auto-contrast
    img = ImageOps.autocontrast(load_img(target_img_paths[11]))
    display(img)
else:
    print(f"Le fichier {target_img_paths[11]} n'existe pas.")


https://www.youtube.com/watch?v=wYiJHFQ6f0Q

In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_resnet50_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained ResNet50 Model """
    resnet50 = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)

    """ Encoder """
    # S is for skip connection
    s1 = resnet50.get_layer("input_layer").output           ## (512 x 512)
    s2 = resnet50.get_layer("conv1_relu").output        ## (256 x 256)
    s3 = resnet50.get_layer("conv2_block3_out").output  ## (128 x 128)
    s4 = resnet50.get_layer("conv3_block4_out").output  ## (64 x 64)

    """ Bridge """
    b1 = resnet50.get_layer("conv4_block6_out").output  ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="ResNet50_U-Net")
    return model

if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_resnet50_unet(input_shape)
    # model.summary()

In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3]) - intersection
    iou = K.mean((intersection + K.epsilon()) / (union + K.epsilon()), axis=0)
    return iou


In [3]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[recall, precision, f1_score, mean_iou])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Supposons que x_train, y_train, x_val, y_val soient vos données d'entraînement et de validation
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    batch_size=8,
    epochs=50,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

In [ ]:
results = model.evaluate(x_val, y_val, batch_size=8)
print("Validation Loss:", results[0])
print("Validation Recall:", results[1])
print("Validation Precision:", results[2])
print("Validation F1 Score:", results[3])
print("Validation Mean IoU:", results[4])
